Step 1: Environment Setup and Data Preparation

In [66]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv"
data = pd.read_csv(url)
data = data.drop(columns=['date'])  # Remove the date column

# Select a subset of features for simplicity
features = ['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'Appliances']
data = data[features]

# Normalize the features
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=features)

# Prepare data for LSTM
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw-1):  # Ensure there's room for the label
        train_seq = torch.tensor(input_data.iloc[i:i+tw].values, dtype=torch.float32)
        train_label = torch.tensor(input_data.iloc[i+tw:i+tw+1].values[:, -1], dtype=torch.float32)  # Assuming last column is target
        inout_seq.append((train_seq, train_label))
    return inout_seq

seq_length = 24  # Using 24 hours of data to predict the next hour
all_data = create_inout_sequences(data_scaled, seq_length)


Step 2: Define DLinear Model 

In [67]:
import torch
import torch.nn as nn

# Define the LSTM-Enhanced Linear Regression Model
class DLinear(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super(DLinear, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out[:, -1, :])  # Only take the output from the last time step
        return predictions

input_size = 7  # Number of features, including 'Appliances' if it is part of the input
hidden_layer_size = 100
output_size = 1

model = DLinear(input_size, hidden_layer_size, output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()




Step 3: Training the Model

In [68]:
def train_model(model, train_data, optimizer, loss_function, epochs=3):
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        for seq, labels in train_data:
            optimizer.zero_grad()  # Clears old gradients from the last step
            seq = seq.reshape(1, seq_length, -1)  # Reshape input to match LSTM expected input
            y_pred = model(seq)  # Forward pass: compute the output
            loss = loss_function(y_pred, labels)  # Compute the loss
            loss.backward()  # Compute gradients
            optimizer.step()  # Update the weights
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')  # Print loss for each epoch

Step 4: Define LSTM-Enhanced Linear Regression Model

In [69]:
class DLinear(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super(DLinear, self).__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        # Ensure input is of the shape [batch_size, seq_length, num_features]
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out.view(-1, self.hidden_layer_size))
        return predictions[-1]


Step 5: Split the Data

In [70]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42)




Step 6: Training and Validation

In [71]:
def train_model(model, train_data, val_data, optimizer, loss_function, epochs=3):
    for epoch in range(epochs):
        model.train()  # Switch to training mode
        train_losses = []
        for seq, labels in train_data:
            optimizer.zero_grad()
            seq = seq.reshape(1, seq_length, -1)
            y_pred = model(seq)
            loss = loss_function(y_pred, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        
        model.eval()  # Switch to evaluation mode for validation
        val_losses = []
        for seq, labels in val_data:
            seq = seq.reshape(1, seq_length, -1)
            y_pred = model(seq)
            val_loss = loss_function(y_pred, labels)
            val_losses.append(val_loss.item())
        
        print(f'Epoch {epoch+1}: Training Loss: {np.mean(train_losses):.4f}, Validation Loss: {np.mean(val_losses):.4f}')


Step 7: Evaluate on Test Data

In [72]:
def evaluate_model(model, test_data, loss_function):
    model.eval()  # Ensure the model is in evaluation mode
    test_losses = []
    with torch.no_grad():
        for seq, labels in test_data:
            seq = seq.reshape(1, seq_length, -1)
            y_pred = model(seq)
            test_loss = loss_function(y_pred, labels)
            test_losses.append(test_loss.item())
    print(f'Average Test Loss: {np.mean(test_losses):.4f}')

Step 8: Run the Training and Evaluation

In [73]:
train_model(model, train_data, test_data, optimizer, loss_function, epochs=3)
evaluate_model(model, test_data, loss_function)

/Users/mustafa/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1: Training Loss: 0.0044, Validation Loss: 0.0041
Epoch 2: Training Loss: 0.0040, Validation Loss: 0.0040
Epoch 3: Training Loss: 0.0039, Validation Loss: 0.0039
Average Test Loss: 0.0039
